# **M1. ACTIVIDAD**
Yuna Chung A01709043

## **Propósito**
Conocer y aplicar una herramienta para la implementación de sistemas multiagentes.

# **Instrucciones**
Para este problema, debereás entregar, **de manera individual**, un informe en PDF que estudie las estadísticas de un robot de limpieza reactivo, así como el enlace al repositorio en Github del código desarrollado para esta actividad. El código debe ajustarse al estilo soclicita en el siguiente documento.

**Dado:**
- Habitación de MxN espacios
- Número de agentes
- Porcentaje de celdas inicialmente sucias
- Tiempo máximo de ejecución

**Realiza la siguiente simulación:**
- Inicializa las celdas sucias (ubicaciones aleatorias)
- Todos los agentes empiezan en la celda [1,1]
- En cada paso de tiempo:
    - Si la celda está sucia, entonces aspira
    - Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda)
- Se ejecuta el tiempo máximo establecido

**Deberás recopilar la siguiente información durante la ejecución:**
- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo)
- Porcentaje de celdas limpias después del termino de la simulación
Número de movimientos realizados por todos los agentes.

Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. Desarrollar un informe con lo observado.

In [ ]:
from mesa import Agent, Model

from mesa.space import MultiGrid

from mesa.time import RandomActivation

from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time as tm

In [ ]:
class CleaningRobot(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

    def step(self):
        if (self.model.floor[self.pos[0]][self.pos[1]] == 1):
            self.clean()
        self.move()

    def clean(self):
        self.model.floor[self.pos[0]][self.pos[1]] = 0

    def move(self):
        close_cells = self.model.grid.get_neighborhood(self.pos, 
                                                       moore=True, 
                                                       include_center=False)
        is_possible = [step for step in close_cells if self.model.grid.is_cell_empty(step)]

        if is_possible:
            self.model.grid.move_agent(self, self.random.choice(is_possible))

In [ ]:
class CleaningModel(Model):
    def __init__(self, width, height, dirty_probability, num_agents):
        self.num_agents = num_agents
        self.initial_time = None
        self.width = width
        self.height = height
        self.grid = MultiGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)
        self.floor = np.zeros((width, height))
        self.datacollector = DataCollector(
            model_reporters = {"Clean": self.count_clean,
                               "Position": self.find_position},
        )

        for i in range(self.num_agents):
            agent = CleaningRobot(i, self)
            self.grid.place_agent(agent, (1,1))
            self.schedule.add(agent)

        for x in range(self.grid.width):
            for y in range(self.grid.height):
                if self.random.random() < dirty_probability:
                    self.floor[x][y] = 1

    def count_clean(self):
        return self.floor.copy()
    
    def find_position(self):
        agent_position = np.zeros((self.width, self.height))
        for agent in self.schedule.agents:
            agent_position[agent.pos[0]][agent.pos[1]] = 1
        return agent_position

    def step(self):
        if self.initial_time is None:
            self.initial_time = tm.time()

        self.datacollector.collect(self)
        self.schedule.step()


In [ ]:
WIDTH = 10
HEIGHT = 10
MAX_STEPS = 100
DIRTY_PROBABILITY = 0.9
NUM_AGENTS = 1

model = CleaningModel(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS)
for i in range(MAX_STEPS):
    model.step()

In [ ]:
all_grids = model.datacollector.get_model_vars_dataframe()

In [ ]:
fig, axis = plt.subplots(figsize=(5, 5))
axis.set_xticks([])
axis.set_yticks([])

data = all_grids.get("Clean") + 3 * all_grids.get("Position")
patch = plt.imshow(data[0], cmap=plt.cm.binary)

def animate(frame):
    patch.set_data(data[frame])

anim = animation.FuncAnimation(fig, animate, frames=MAX_STEPS)

In [ ]:
# anim

In [ ]:
def get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS):
    model = CleaningModel(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS)
    for i in range(MAX_STEPS):
        model.step()

    end_time = tm.time()

    cleaning_time = float("{:.2f}".format(end_time - model.initial_time))
    dirty_cells = model.datacollector.get_model_vars_dataframe().iloc[MAX_STEPS - 1][0].sum()

    if dirty_cells == 0:
        dirty_cells_percentage = 0
    else:
        dirty_cells_percentage = float("{:.2f}".format(dirty_cells / (WIDTH * HEIGHT) * 100))
    
    
    return cleaning_time, dirty_cells_percentage

## **Escenario 1**

- 1 agente
- 90% de celdas sucias

In [ ]:
WIDTH = 100
HEIGHT = 100
MAX_STEPS = 100
DIRTY_PROBABILITY = 0.9
NUM_AGENTS = 1

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("100 iteraciones: \n")
print("Tiempo tardado para limpiar: ", data[0], " segundos")
print("Porcentaje de celdas sucias: ", data[1], "% \n")

MAX_STEPS = 1000

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("1000 iteraciones: \n")
print("Time to clean: ", data[0], " seconds")
print("Dirty cells percentage: ", data[1], "% \n")

MAX_STEPS = 10000

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("10000 iteraciones: \n")
print("Time to clean: ", data[0], " seconds")
print("Dirty cells percentage: ", data[1], "% \n")


## **Escenario 2**

- 2 agentes
- 90% de celdas sucias

In [ ]:
WIDTH = 100
HEIGHT = 100
MAX_STEPS = 100
DIRTY_PROBABILITY = 0.9
NUM_AGENTS = 2

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("100 iteraciones: \n")
print("Tiempo tardado para limpiar: ", data[0], " segundos")
print("Porcentaje de celdas sucias: ", data[1], "% \n")

MAX_STEPS = 1000

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("1000 iteraciones: \n")
print("Time to clean: ", data[0], " seconds")
print("Dirty cells percentage: ", data[1], "% \n")

MAX_STEPS = 10000

data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, NUM_AGENTS, MAX_STEPS)

print("10000 iteraciones: \n")
print("Time to clean: ", data[0], " seconds")
print("Dirty cells percentage: ", data[1], "% \n")


## **ESTADÍSTICA POR DIFERENTES NÚMEROS DE AGENTES**

In [ ]:
WIDTH = 100
HEIGHT = 100
MAX_STEPS = 10000
DIRTY_PROBABILITY = 0.9
NUM_AGENTS = [1,2,3,4,5,6,7,8,9,10]

for i in NUM_AGENTS:
    data = get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, i, MAX_STEPS)
    print("Número de agentes: ", i)
    print("Tiempo tardado para limpiar: ", data[0], " segundos")
    print("Porcentaje de celdas sucias: ", data[1], "% \n")

In [ ]:
data = []
for i in NUM_AGENTS:
    data.append(get_model_statics(WIDTH, HEIGHT, DIRTY_PROBABILITY, i, MAX_STEPS)[1])

plt.plot(NUM_AGENTS, data)
plt.xlabel('Número de agentes')
plt.ylabel('Porcentaje de celdas sucias')
plt.show()